# Import Library

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import ast
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Data Understanding

## Load Data

In [2]:
df = pd.read_csv('goodreads_data.csv')

## Exploratory Data Analysis

### Check Data

In [3]:
df.head()

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


In [4]:
df.shape

(10000, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  9923 non-null   object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  object 
 7   URL          10000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


In [6]:
df.isnull().sum()

Unnamed: 0      0
Book            0
Author          0
Description    77
Genres          0
Avg_Rating      0
Num_Ratings     0
URL             0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
df['Genres'].unique()

array(["['Classics', 'Fiction', 'Historical Fiction', 'School', 'Literature', 'Young Adult', 'Historical']",
       "['Fantasy', 'Fiction', 'Young Adult', 'Magic', 'Childrens', 'Middle Grade', 'Classics']",
       "['Classics', 'Fiction', 'Romance', 'Historical Fiction', 'Literature', 'Historical', 'Audiobook']",
       ...,
       "['Dystopia', 'Science Fiction', 'Post Apocalyptic', 'Paranormal', 'Fantasy']",
       "['Fiction', 'Horror', 'Dystopia', 'Coming Of Age']",
       "['New Adult', 'Romance', 'Contemporary Romance', 'Contemporary']"],
      dtype=object)

**Insight**
- Dataset berikut memiliki baris 10000 dan 8 kolom
- Pada kolom `Num_Ratings` masih memiliki tipe data yang object, yang harusnya kolom tersebut memiliki tipe data float
- Terdapat missing value pada kolom `Description` yang nantinya akan ditangani dengan mengisi missing valuenya. Dan untuk duplicated data tidak ditemukan disini.
- Pada kolom `Genres` memiliki banyak distribsi dalam satu kolom. Kolom tersebut memiliki format string, ini perlu dilakukan perubahan menjadi list agar memudahkan ketika pengolahan dan perhitungan kesamaan genre.

# Data Preparation

### Preprocessing Data

In [9]:
# Menghapus kolom yang tidak diperlukan
df_cleaned = df.drop(columns=['Unnamed: 0', 'URL'])

# Menghapus koma pada kolom
df_cleaned['Num_Ratings'] = df_cleaned['Num_Ratings'].str.replace(',', '')

# Mengonversi kolom 'Num_Ratings' menjadi float64
df_cleaned['Num_Ratings'] = pd.to_numeric(df_cleaned['Num_Ratings'], errors='coerce')

# Mengisi nilai kosong di kolom 'Description' dengan string kosong
df_cleaned['Description'].fillna('No Description', inplace=True)

# Mengonversi kolom Genres yang berupa string menjadi list
df_cleaned['Genres'] = df_cleaned['Genres'].apply(lambda x: x.strip("[]").replace("'", "").split(',') if isinstance(x, str) else x)

# Membersihkan genre untuk memastikan kesesuaian format (menggabungkan list menjadi string)
df_cleaned['Genres'] = df_cleaned['Genres'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Menggabungkan kolom 'Genres', 'Description', 'Author', dan 'Book' menjadi satu kolom teks
df_cleaned['combined'] = df_cleaned['Genres'] + ' ' + df_cleaned['Description'] + ' ' + df_cleaned['Author'] + ' ' + df_cleaned['Book']

# Melihat data yang telah digabungkan
print(df_cleaned[['Book', 'combined']].head())

                                                Book  \
0                              To Kill a Mockingbird   
1  Harry Potter and the Philosopher’s Stone (Harr...   
2                                Pride and Prejudice   
3                          The Diary of a Young Girl   
4                                        Animal Farm   

                                            combined  
0  Classics  Fiction  Historical Fiction  School ...  
1  Fantasy  Fiction  Young Adult  Magic  Children...  
2  Classics  Fiction  Romance  Historical Fiction...  
3  Classics  Nonfiction  History  Biography  Memo...  
4  Classics  Fiction  Dystopia  Fantasy  Politics...  


C:\Users\dumbw\AppData\Local\Temp\ipykernel_19156\1705519255.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Description'].fillna('No Description', inplace=True)


**Insight Preprocessing Data**

1. **Menghapus Kolom yang Tidak Diperlukan:**

   * Kolom **`Unnamed: 0`** dan **`URL`** dihapus karena tidak memberikan informasi relevan untuk analisis buku atau rekomendasi. Proses ini membantu mengurangi kompleksitas dataset dan memastikan fokus hanya pada data yang benar-benar dibutuhkan untuk analisis.

2. **Menghapus Koma pada `Num_Ratings`:**

   * Kolom **`Num_Ratings`** mengandung koma dalam nilai numeriknya, yang dapat mengganggu konversi tipe data. Dengan menghapus koma, data menjadi lebih bersih dan siap untuk dikonversi menjadi **float64**, yang memungkinkan kita melakukan analisis numerik yang lebih tepat, seperti pengurutan atau pengelompokan berdasarkan rating.

3. **Mengonversi `Num_Ratings` ke Tipe Numerik:**

   * Mengubah **`Num_Ratings`** menjadi tipe **float64** memastikan bahwa data bisa dihitung dan dianalisis dengan benar, seperti melakukan pengurutan berdasarkan rating atau melakukan evaluasi terhadap popularitas buku berdasarkan jumlah ulasan.

4. **Mengisi Nilai Kosong di `Description`:**

   * Nilai kosong pada kolom **`Description`** diisi dengan **"No Description"** untuk menjaga integritas data. Ini mencegah terjadinya missing values yang dapat mengganggu pemodelan dan analisis lebih lanjut, memastikan bahwa seluruh dataset lengkap dan dapat diproses tanpa masalah.

5. **Mengonversi `Genres` Menjadi List:**

   * Kolom **`Genres`** yang awalnya berbentuk string diubah menjadi **list** untuk memudahkan analisis lebih lanjut, terutama dalam menghitung kesamaan genre antar buku. Mengonversi genre menjadi list juga memungkinkan pemrosesan yang lebih efisien ketika mencari buku dengan genre yang serupa.

6. **Membersihkan dan Menggabungkan `Genres`:**

   * **Genres** yang sebelumnya berupa list kini diubah menjadi string yang dipisahkan oleh spasi untuk memastikan format yang konsisten. Proses ini menjamin bahwa data genre dapat diproses dengan benar dalam analisis kesamaan teks dan meningkatkan keterbacaan dataset.

7. **Menggabungkan Kolom `Genres`, `Description`, `Author`, dan `Book`:**

   * Menggabungkan kolom-kolom **`Genres`**, **`Description`**, **`Author`**, dan **`Book`** ke dalam satu kolom **`combined`** memungkinkan untuk menghitung kemiripan antar buku berdasarkan keseluruhan konteks (genre, deskripsi, penulis, dan judul buku). Kolom **`combined`** ini merupakan fitur utama untuk model rekomendasi berbasis konten yang akan digunakan untuk menghitung kemiripan antar buku.

Proses preprocessing yang dilakukan ini memastikan dataset siap digunakan dalam model **Content-Based Filtering** untuk memberikan rekomendasi buku. Dengan membersihkan dan memformat ulang data, dataset menjadi lebih konsisten dan terstruktur, memudahkan dalam perhitungan kemiripan antara buku berdasarkan konten yang relevan.

### Feature Engineering

In [10]:
# Menggunakan TfidfVectorizer untuk menghitung representasi TF-IDF dari teks yang telah digabungkan
tfidf = TfidfVectorizer(stop_words='english')

# Menghitung matriks TF-IDF
tfidf_matrix = tfidf.fit_transform(df_cleaned['combined'])

In [11]:
# Menggunakan CountVectorizer untuk menghitung representasi biner dari teks (deskripsi buku)
count_vectorizer = CountVectorizer(stop_words='english')
count_matrix = count_vectorizer.fit_transform(df_cleaned['combined'])

**Insight Feature Engginering**

Untuk meningkatkan performa sistem rekomendasi, digunakan teknik **feature engineering** untuk mengubah teks menjadi representasi numerik yang dapat digunakan dalam perhitungan kemiripan antar buku. Dua teknik yang digunakan adalah **TfidfVectorizer** dan **CountVectorizer**.

* **TfidfVectorizer**:
  `TfidfVectorizer` digunakan untuk mengonversi teks menjadi representasi numerik dengan memperhitungkan **frekuensi kata** dalam dokumen dan **invers frekuensi** kata tersebut dalam koleksi dokumen. Teknik ini memberikan bobot lebih pada kata-kata yang jarang namun relevan, sehingga membantu dalam menangkap kata-kata penting dalam konteks deskripsi dan genre buku.

* **CountVectorizer**:
  ``CountVectorizer`` mengubah teks menjadi representasi numerik berdasarkan **frekuensi kemunculan kata** dalam dokumen. Teknik ini efektif untuk mengukur kesamaan berdasarkan kata-kata yang muncul dalam teks, terutama dalam deskripsi dan genre buku.

---

**Alasan Penggunaan**:

* **TfidfVectorizer** membantu menangkap kata-kata penting dan relevan dalam deskripsi buku dan genre.
* **CountVectorizer** digunakan untuk mengukur kesamaan kata yang lebih eksplisit dalam teks, khususnya untuk elemen genre atau deskripsi yang muncul berulang.

Kedua teknik ini meningkatkan kemampuan model untuk menghitung kemiripan antar buku dan memperbaiki akurasi rekomendasi berbasis konten.

# Modeling

### Cousine Simmilarity

In [12]:
# Menghitung cosine similarity antar buku menggunakan TF-IDF matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Fungsi untuk rekomendasi berdasarkan Cosine Similarity
def recommend_books_cosine(query, cosine_sim=cosine_sim, top_n=5):
    query_tfidf = tfidf.transform([query])
    cosine_sim_query = cosine_similarity(query_tfidf, tfidf_matrix)
    
    sim_scores = list(enumerate(cosine_sim_query[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    sim_scores = sim_scores[0:top_n]
    book_indices = [i[0] for i in sim_scores]
    
    recommended_books = df_cleaned[['Book', 'Author', 'Genres', 'Avg_Rating', 'Num_Ratings']].iloc[book_indices]
    recommended_books['Num_Ratings'] = recommended_books['Num_Ratings'].apply(pd.to_numeric, errors='coerce')
    recommended_books = recommended_books.sort_values(by='Num_Ratings', ascending=False)
    
    return recommended_books.head(top_n)

# Menggunakan contoh query
recommended_books_cosine = recommend_books_cosine('fiction')
print("Top 5 recommended books based on Cosine Similarity:")
print(recommended_books_cosine)

Top 5 recommended books based on Cosine Similarity:
                                                 Book           Author  \
8508                       Virtual Light (Bridge, #1)   William Gibson   
4362              Forever Peace (The Forever War, #3)     Joe Haldeman   
5706                      Dear and Glorious Physician  Taylor Caldwell   
9616  The Enigma of Arrival: A Novel in Five Sections     V.S. Naipaul   
9511                                Collected Fiction      Ruskin Bond   

                                                 Genres  Avg_Rating  \
8508  Science Fiction  Cyberpunk  Fiction  Science F...        3.87   
4362  Science Fiction  Fiction  Hugo Awards  War  Sc...        3.75   
5706  Historical Fiction  Fiction  Religion  Histori...        4.33   
9616  Fiction  Literature  Nobel Prize  Novels  Hist...        3.70   
9511                      Fiction  India  Short Stories        4.50   

      Num_Ratings  
8508        24657  
4362        20566  
5706         725

### Jaccard Distance

In [13]:
# Fungsi untuk menghitung Jaccard Distance
def jaccard_distance(vec1, vec2):
    intersection = np.sum(np.minimum(vec1, vec2))  # Jumlah kata yang ada di kedua buku
    union = np.sum(np.maximum(vec1, vec2))  # Jumlah kata yang ada di salah satu buku
    return 1 - intersection / union  # Jaccard Distance


# Fungsi untuk rekomendasi berdasarkan Jaccard Distance
def recommend_books_jaccard(query, count_matrix=count_matrix, top_n=5):
    query_vec = count_vectorizer.transform([query]).toarray().flatten()  # Mengubah query menjadi vektor biner
    jaccard_scores = []
    
    # Menghitung Jaccard Distance antara query dan setiap buku dalam dataset
    for i in range(count_matrix.shape[0]):
        book_vec = count_matrix[i].toarray().flatten()
        score = jaccard_distance(query_vec, book_vec)
        jaccard_scores.append((i, score))
    
    # Mengurutkan berdasarkan skor Jaccard Distance (semakin kecil nilai, semakin mirip)
    jaccard_scores = sorted(jaccard_scores, key=lambda x: x[1])
    
    # Memilih top_n buku teratas
    top_books_indices = [i[0] for i in jaccard_scores[:top_n]]
    recommended_books = df_cleaned[['Book', 'Author', 'Genres', 'Avg_Rating', 'Num_Ratings']].iloc[top_books_indices]
    recommended_books['Num_Ratings'] = recommended_books['Num_Ratings'].apply(pd.to_numeric, errors='coerce')
    recommended_books = recommended_books.sort_values(by='Num_Ratings', ascending=False)
    
    return recommended_books.head(top_n)

# Menggunakan contoh query
recommended_books_jaccard = recommend_books_jaccard('fiction')
print("Top 5 recommended books based on Jaccard Distance:")
print(recommended_books_jaccard)


Top 5 recommended books based on Jaccard Distance:
                                      Book          Author  \
8108  The Shipping News by E. Annie Proulx        BookRags   
2362                       SHADOW PANTHEON  Eric Nierstedt   
7413          A Taste for Green Tangerines   Barbara Bisco   
7283                         Making Amends  D.J. Callaghan   
5763                        Chasing Dreams  Aaron Jennings   

                         Genres  Avg_Rating  Num_Ratings  
8108  Fiction  Literary Fiction        4.33          566  
2362      Fiction  Contemporary        4.28          146  
7413                    Fiction        3.24           71  
7283             Fiction  Drama        4.20           51  
5763            Travel  Fiction        3.24           37  


**Insight Modeling**

### **1. Cosine Similarity:**

  * **Cosine Similarity**: Menghitung **Cosine Similarity** antara vektor vektor teks untuk mengukur seberapa mirip dua buku berdasarkan kesamaan dalam deskripsi, genre, penulis, dan judul.

* **Hasil yang Diperoleh:**

  * Rekomendasi yang diberikan berdasarkan **Cosine Similarity** menunjukkan buku yang memiliki kesamaan **kontekstual** dalam hal deskripsi, genre, dan penulis.

  * Buku yang direkomendasikan:

    * **Virtual Light** oleh William Gibson
    * **Forever Peace** oleh Joe Haldeman
    * **Dear and Glorious Physician** oleh Taylor Caldwell
    * **The Enigma of Arrival** oleh V.S. Naipaul
    * **Collected Fiction** oleh Ruskin Bond

  * **Rating dan Popularitas**: Buku yang lebih banyak di-review (seperti **Virtual Light**) lebih tinggi dalam rekomendasi karena pengurutan berdasarkan **`Num_Ratings`**.

---

### **2. Jaccard Distance:**
  
  * **Jaccard Distance**: Menghitung **Jaccard Distance** untuk mengukur perbedaan antara dua buku. Jaccard mengukur kesamaan antar set dengan membandingkan jumlah elemen yang sama terhadap jumlah elemen total yang ada di kedua set.

* **Hasil yang Diperoleh:**

  * Rekomendasi yang diberikan berdasarkan **Jaccard Distance** menunjukkan buku yang memiliki kesamaan **kata kunci** atau elemen **spesifik** dalam deskripsi dan genre.

  * Buku yang direkomendasikan:

    * **The Shipping News** oleh E. Annie Proulx
    * **SHADOW PANTHEON** oleh Eric Nierstedt
    * **A Taste for Green Tangerines** oleh Barbara Bisco
    * **Making Amends** oleh D.J. Callaghan
    * **Chasing Dreams** oleh Aaron Jennings

  * **Rating dan Popularitas**: Buku dengan rating tinggi (misalnya **The Shipping News**) lebih tinggi dalam rekomendasi karena pengurutan berdasarkan **`Num_Ratings`**.

---

### **Kesimpulan:**

* **Cosine Similarity** menggunakan representasi **TF-IDF** untuk menghitung kemiripan berdasarkan kesamaan **kontekstual** dalam deskripsi, genre, dan penulis.
* **Jaccard Distance** berfokus pada **kesamaan kata** dan **genre** dalam deskripsi buku, yang lebih menekankan pada elemen spesifik yang ada pada kedua buku.


# Evaluation

In [14]:
# Fungsi untuk menghitung Precision at K
def precision_at_k(recommended_books, relevant_books, k):
    """
    Menghitung Precision at K
    - recommended_books: Daftar buku yang direkomendasikan
    - relevant_books: Daftar buku relevan yang sesuai dengan preferensi pengguna
    - k: Jumlah buku yang dipertimbangkan dalam rekomendasi
    """
    recommended_at_k = recommended_books[:k]
    relevant_at_k = [book for book in recommended_at_k if book in relevant_books]
    precision = len(relevant_at_k) / k
    return precision

# Misalnya, relevan buku adalah buku dengan rating tinggi (> 4)
relevant_books_cosine = recommended_books_cosine[recommended_books_cosine['Avg_Rating'] > 4]['Book'].tolist()
relevant_books_jaccard = recommended_books_jaccard[recommended_books_jaccard['Avg_Rating'] > 4]['Book'].tolist()

# Tentukan k (jumlah rekomendasi)
k = 5

# Menghitung Precision at K untuk Cosine Similarity
precision_cosine = precision_at_k(recommended_books_cosine['Book'].tolist(), relevant_books_cosine, k)
print(f"Precision at K for Cosine Similarity: {precision_cosine}")

# Menghitung Precision at K untuk Jaccard Distance
precision_jaccard = precision_at_k(recommended_books_jaccard['Book'].tolist(), relevant_books_jaccard, k)
print(f"Precision at K for Jaccard Distance: {precision_jaccard}")

Precision at K for Cosine Similarity: 0.4
Precision at K for Jaccard Distance: 0.6


### **Insight dari Evaluasi Precision at K:**

Dari hasil evaluasi **Precision at K** :

* **Precision at K for Cosine Similarity: 0.4**
* **Precision at K for Jaccard Distance: 0.6**

Ini menunjukkan perbandingan antara dua metode (Cosine Similarity dan Jaccard Distance) dalam memberikan rekomendasi yang relevan berdasarkan **rating tinggi** (misalnya rating > 4).

### **Penjelasan Lebih Lanjut:**

1. **Precision at K for Cosine Similarity (0.4):**

   * **Interpretasi:** Hanya **40%** dari **top-5 rekomendasi** berdasarkan **Cosine Similarity** yang relevan dengan preferensi pengguna (misalnya, buku dengan rating lebih tinggi atau genre yang sesuai).
   * **Kemungkinan Penyebab:**

     * **Cosine Similarity** mengukur **kemiripan konteks** secara keseluruhan, tetapi terkadang bisa memberi hasil yang kurang relevan jika **konteks teksnya mirip** namun tidak berhubungan dengan **preferensi rating tinggi** atau **genre** tertentu.
     * Ini bisa terjadi jika **relevansi** yang didasarkan pada **rating tinggi** atau **genre yang cocok** tidak cukup berhubungan dengan teks buku (misalnya, genre atau deskripsi).

2. **Precision at K for Jaccard Distance (0.6):**

   * **Interpretasi:** **60%** dari **top-5 rekomendasi** berdasarkan **Jaccard Distance** relevan dengan preferensi pengguna (misalnya, buku dengan rating lebih tinggi atau genre yang sesuai).
   * **Kemungkinan Penyebab:**

     * **Jaccard Distance** lebih fokus pada **kesamaan elemen spesifik** (kata atau genre), yang membuatnya lebih **berorientasi pada kesamaan langsung** dalam deskripsi buku.
     * Karena **Jaccard Distance** mengukur **kesamaan kata atau genre**, sistem ini mungkin lebih efektif dalam menangkap **kesamaan yang lebih langsung** dengan **preferensi pengguna** yang lebih sederhana, seperti **rating tinggi** dan genre.

---

### **Implikasi dari Hasil Evaluasi:**

1. **Cosine Similarity:**

   * **Kelebihan:** Cosine Similarity cocok ketika kita ingin memberikan rekomendasi berdasarkan **kesamaan konteks keseluruhan** (deskripsi, genre, penulis, dll). Namun, jika pengguna lebih tertarik pada buku dengan **rating tinggi** atau **genre tertentu**, sistem ini mungkin tidak seefektif Jaccard Distance.
   * **Peningkatan:** Kamu bisa **menambahkan bobot** untuk genre atau rating agar sistem lebih menekankan pada buku dengan **rating lebih tinggi** dan **kesesuaian genre**.

2. **Jaccard Distance:**

   * **Kelebihan:** Jaccard lebih menekankan pada **kesamaan elemen langsung**, seperti kata kunci atau genre dalam deskripsi buku. Dengan demikian, **precision** lebih tinggi karena rekomendasi lebih relevan dengan **kesamaan kata** yang spesifik.
   * **Peningkatan:** Agar hasil lebih baik, kamu bisa **memperkenalkan bobot** berdasarkan **rating** atau **jumlah rating**, memastikan buku yang lebih populer dan relevan tetap mendapatkan prioritas meskipun mungkin tidak memiliki kesamaan teks yang terlalu tinggi.

---

### **Kesimpulan:**

* **Jaccard Distance** memberikan hasil yang lebih baik dalam hal relevansi rekomendasi, namun kedua metode ini dapat digabungkan atau disesuaikan lebih lanjut untuk meningkatkan kualitas rekomendasi.
* **Jaccard Distance** memberikan hasil yang lebih baik dalam hal **Precision at K**, karena ia lebih fokus pada **kesamaan kata atau genre** yang lebih jelas dan langsung.
* **Cosine Similarity** meskipun bermanfaat untuk memahami **kesamaan konteks keseluruhan**, mungkin tidak cukup efektif dalam menghasilkan **rekomendasi yang relevan** jika kita lebih mengutamakan **rating tinggi** atau **genre tertentu**.
